***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-fb6c  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 24 17:29 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket_for_index' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [9]:
parquetFile = spark.read.parquet(*paths)
body_id_rdd = parquetFile.select("text", "id").rdd
anchor_id_rdd = parquetFile.select("anchor_text", "id").rdd

In [11]:
print(body_id_rdd.count())

24/02/25 11:41:42 WARN TaskSetManager: Lost task 120.0 in stage 2.0 (TID 182) (cluster-fb6c-w-2.c.irass3-413914.internal executor 6): org.apache.spark.sql.execution.QueryExecutionException: Encountered error while reading file gs://bucket_for_index/project_pageRank/pageRank.pkl. Details: 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:730)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:283)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:573)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.Buffer

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 120 in stage 2.0 failed 4 times, most recent failure: Lost task 120.3 in stage 2.0 (TID 188) (cluster-fb6c-w-2.c.irass3-413914.internal executor 5): org.apache.spark.sql.execution.QueryExecutionException: Encountered error while reading file gs://bucket_for_index/project_pageRank/pageRank.pkl. Details: 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:730)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:283)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:573)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_

In [ ]:
print(achor_id_rdd.count())

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

# Building an Anchor Index for the Project #

In [8]:
from pyspark.sql import functions as F
from pyspark.ml.feature import HashingTF, IDF

import math

def PRINT(text) -> None: print(f'{"-"*80}\n{text}\n{"-"*80}')

## Calculate Documents L2 Normalization ##

In the next step we calculate L2 Norm for each document text.

The ending result will be dictionary which maps -> (key, value) to (doc_id, doc_l2_norm_value)

We will need that in order to preform CosinSimilarity in the next steps of the project

In [10]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                            "may", "first", "see", "history", "people", "one", "two",
                            "part", "thumb", "including", "second", "following",
                            "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()

In [11]:
def tokenize(text, to_stem):
    clean_text = []
    text = text.lower()
    tokens = [token.group() for token in RE_WORD.finditer(text)]
    for token in tokens:
        if token not in all_stopwords:
            if to_stem:
                token = stemmer.stem(token)
                clean_text.append(token)
    return clean_text

In [12]:
text = "This is some kind OF test hello test test kind"

text = tokenize(text, True)
print(text)


['kind', 'test', 'hello', 'test', 'test', 'kind']


In [13]:
from collections import Counter

def l2_norm(text):
    # Count the occurrences of each unique word
    word_counts = Counter(text)
    # Get the counts of each unique word as a list
    counts_list = [word_counts[word]**2 for word in set(text)]
    l2_sum = 0
    for word_num in counts_list:
        l2_sum += word_num
    l2 = math.sqrt(l2_sum)
    #l2_norm = math.sqrt(sum(x**2 for x in counts_list))
    return l2

text = "This is some hell of a way just to add add add this function"
text_tok = tokenize(text, True)
print(text_tok)
print(l2_norm(text_tok))


['hell', 'way', 'add', 'add', 'add', 'function']
3.4641016151377544


In [17]:
parquetFile = spark.read.parquet(*paths)
rdd_ = parquetFile.select("text", "id").rdd

In [33]:
doc_norm = {}

In [21]:
def calculate_doc_l2_norm(row):
    doc_id = row['id']
    text = row['text']
    tok_text = tokenize(text, True)
    return (doc_id, l2_norm(tok_text))

doc_norm_rdd = rdd_.map(calculate_doc_l2_norm)

In [22]:
doc_norm = dict(doc_norm_rdd.collect())

Exception in thread "serve RDD 17" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [26]:
base_dir='project_pkl' 
doc_l2_norm='doc_l2_norm'
bucket_name='bucket_for_index'

In [27]:
path = str(Path(base_dir) / f'{doc_l2_norm}.pkl')
bucket = None if bucket_name is None else get_bucket(bucket_name)

Path(base_dir).mkdir(parents=True, exist_ok=True)

blob = bucket.blob(path)
pickle.dump(doc_norm, open(path, 'wb'))
blob.upload_from_filename(path)

In [29]:
for key, value in list(doc_norm.items())[:10]:
    PRINT(f"Key: {key}, Value: {value}")

--------------------------------------------------------------------------------
Key: 4045403, Value: 125.50298801223818
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045413, Value: 28.319604517012593
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045419, Value: 38.57460304397182
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045426, Value: 22.891046284519195
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045432, Value: 46.119410230400824
------------------------------------------------------------------------

## Explore the RDD Dataset ##

In [22]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs_test = parquetFile.rdd

# Convert RDD to DataFrame to view column names
doc_text_pairs_df = doc_text_pairs_test.toDF()

# Print column names
print(doc_text_pairs_df.columns)

['id', 'title', 'text', 'anchor_text']


In [10]:
doc_text_pairs_df.show()

+-------+--------------------+--------------------+--------------------+
|     id|               title|                text|         anchor_text|
+-------+--------------------+--------------------+--------------------+
|4045403|Foster Air Force ...|'''Foster Air For...|[{1176764, Tactic...|
|4045413|     Torino Palavela|'''Palavela''', f...|[{77743, 2006 Win...|
|4045419|   Mad About the Boy|"'''Mad About the...|[{34028256, Joyce...|
|4045426|       Shayne Breuer|'''Shayne Breuer'...|[{1838386, Woodvi...|
|4045432|         Parantaka I|'''Parantaka Chol...|[{1511716, Aditya...|
|4045456|Arundel (UK Parli...|'''Arundel''' was...|[{4665376, Arunde...|
|4045466|     Andrew Martinez|'''Luis Andrew Ma...|[{4860, Berkeley,...|
|4045471|    Vancouver VooDoo|The '''Vancouver ...|[{32706, Vancouve...|
|4045479|     Invisible plane|The '''Invisible ...|[{2260539, Ross A...|
|4045516|    Shopping channel|'''Shopping chann...|[{592899, special...|
|4045519|      Turgay (river)|The '''Turgay''' ...|

## Import Inverted Index .py File ##

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

## Pipeline Code to Generate the Index ##

In [13]:
stemmer = PorterStemmer()
# Getting tokens from the text while removing punctuations.
def filter_tokens(tokens, tokens2remove=None, use_stemming=False):
  ''' 
  The function takes a list of tokens, filters out `tokens2remove` and
      stem the tokens using `stemmer`.
  Parameters:
  -----------
  tokens: list of str.
    Input tokens.
  tokens2remove: frozenset.
    Tokens to remove (before stemming).
  use_stemming: bool.
    If true, apply stemmer.stem on tokens.
  Returns:
  --------
  list of tokens from the text.
  '''
  # YOUR CODE HERE
  if tokens2remove != None:
    tokens = [token for token in tokens if token not in tokens2remove]
  if use_stemming:
    tokens = [stemmer.stem(token) for token in tokens]
  return tokens

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

stemmer = PorterStemmer()

NUM_BUCKETS = 124

def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS


def word_count(text, id):
  ''' 
  Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
    text: str ,Text of one document
    id: int ,Document id
  Returns: List of tuples, A list of (token, (doc_id, tf)) pairs 
  '''

  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  
  c = Counter([tok for tok in tokens if (tok not in all_stopwords)])
  return [(item[0],(id,item[1])) for item in c.items()]

def word_count_with_stemming(text, id):
  ''' 
  Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
    text: str ,Text of one document
    id: int ,Document id
  Returns: List of tuples, A list of (token, (doc_id, tf)) pairs 
  '''

  tokens = [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower())]
  
  c = Counter([tok for tok in tokens if (tok not in all_stopwords)])
  return [(item[0],(id,item[1])) for item in c.items()]

def reduce_word_counts(unsorted_pl):
  ''' 
  Returns a sorted posting list by wiki_id.
  Parameters: unsorted_pl: list of (wiki_id, tf) tuples 
  Returns: A sorted posting list.
  '''
  return sorted(unsorted_pl)

def calculate_df(postings):
  ''' 
  Takes a posting list RDD and calculate the df for each token.
  Parameters: postings: RDD ,An RDD where each element is a (token, posting_list) pair.
  Returns:RDD ,An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda x: (x[0],len(x[1])))


def partition_postings_and_write(postings, base_dir_, bucket_name_):
  ''' 
  A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning done by using `token2bucket`.
  Parameters:
  
  base_dir : string - Name for index directory
  bucket_name : string - Name of the bucket we want to store our inverted index directory (base_dir)
  -----------
    postings: RDD , An RDD where each item is a (w, posting_list) pair.
  Returns: RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to.
  '''
  return postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey().map(lambda x: InvertedIndex.write_a_posting_list(x,
                                                                                                                         base_dir= base_dir_,
                                                                                                                         bucket_name=bucket_name_))

## Generate Inverted Index For Title & Id Pairs ##

In [17]:
title_id_pairs = parquetFile.select("title", "id").rdd

In [18]:
count_pairs = title_id_pairs.count()
print(f'Number of title & id pairs -> {count_pairs}')

Number of title & id pairs -> 6348910


### Generate Posting Lists for Title Index & Save them ###

In [22]:
# time the index creation time
t_start = time()

# word counts map
word_counts = title_id_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered,
                                 base_dir_='title_index_directory',
                                 bucket_name_='inverted_indexes_bucket').collect()

index_const_time = time() - t_start

In [23]:
# test index construction time
assert index_const_time < 60*120

In [24]:
blobs_test = client.list_blobs(bucket_or_name='inverted_indexes_bucket',
                              prefix='title_index_directory')
count = 0
# Iterate over the blobs
for blob in blobs_test:
    if count == 10: break
    count+=1
    print(blob.name)
    
PRINT('Done')

project_final_indexes/title_index.pkl
title_index_directory/0_000.bin
title_index_directory/0_posting_locs.pickle
title_index_directory/100_000.bin
title_index_directory/100_posting_locs.pickle
title_index_directory/101_000.bin
title_index_directory/101_posting_locs.pickle
title_index_directory/102_000.bin
title_index_directory/102_posting_locs.pickle
title_index_directory/103_000.bin
--------------------------------------------------------------------------------
Done
--------------------------------------------------------------------------------


In [36]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)

for blob in client.list_blobs(bucket_or_name='inverted_indexes_bucket',
                             prefix='title_index_directory'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
PRINT('Done.')

--------------------------------------------------------------------------------
Done.
--------------------------------------------------------------------------------


Putting it all together

In [42]:
# Create inverted index instance
inverted = InvertedIndex()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs

# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

# write the global stats out
inverted.write_index(base_dir='project_final_indexes', 
                     name='title_index',
                     bucket_name='inverted_indexes_bucket')

PRINT('Done')

--------------------------------------------------------------------------------
Done
--------------------------------------------------------------------------------


### Visualize the Index ###

In [39]:
def load_index(base_dir, name, bucket_name):
    # Initialize the client
    client = storage.Client()
    
    # Get the bucket
    bucket = client.get_bucket(bucket_name)
    
    # Define the path to the index file in the bucket
    index_path = f'{base_dir}/{name}.pkl'
    
    # Get the blob (file) from the bucket
    blob = bucket.blob(index_path)
    
    # Download the blob into memory
    index_data = blob.download_as_string()
    
    # Load the index from the downloaded data
    inverted_index = pickle.loads(index_data)
    
    return inverted_index

In [70]:
inverted_index = load_index(base_dir='project_final_indexes', 
                            name='title_index',
                            bucket_name='inverted_indexes_bucket')

In [71]:
posting_locs = inverted_index.posting_locs
df_data = inverted_index.df

# Convert the posting_locs dictionary to a DataFrame
posting_locs_df = pd.DataFrame.from_dict(posting_locs, orient='index', columns=['Posting List'])

# Convert the df_data dictionary to a DataFrame
df_data_df = pd.DataFrame.from_dict(df_data, orient='index', columns=['Document Frequency'])

In [72]:
df_data_df_sorted = df_data_df.sort_values(by='Document Frequency', ascending=True)

In [73]:
PRINT(f'Data Frame shape -> {df_data_df_sorted.shape}')

--------------------------------------------------------------------------------
Data Frame shape -> (26981, 1)
--------------------------------------------------------------------------------


In [74]:
df_data_df_sorted.head(10)

Document Frequency
skyhawks                    51
ios                         51
makino                      51
405                         51
sewall                      51
euchromius                  51
liberator                   51
yoder                       51
wasserman                   51
pani                        51

In [54]:
PRINT('Done with Title Index !')

--------------------------------------------------------------------------------
Done with Title Index !
--------------------------------------------------------------------------------


## Generate Inverted Index for Title & ID Pairs using Stemming ##

In [30]:
# time the index creation time
t_start = time()

# word counts with stemming map
word_counts_with_stemming = title_id_pairs.flatMap(lambda x: word_count_with_stemming(x[0], x[1]))
postings = word_counts_with_stemming.groupByKey().mapValues(reduce_word_counts)

# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered,
                                 base_dir_='title_stem_index_directory',
                                 bucket_name_='inverted_indexes_bucket').collect()

index_const_time = time() - t_start

In [35]:
blobs_test = client.list_blobs(bucket_or_name='inverted_indexes_bucket',
                              prefix='title_stem_index_directory')
count = 0
# Iterate over the blobs
for blob in blobs_test:
    if count == 10:break
    count+=1
    print(blob.name)
    
PRINT('Done')

title_stem_index_directory/0_000.bin
title_stem_index_directory/0_posting_locs.pickle
title_stem_index_directory/100_000.bin
title_stem_index_directory/100_posting_locs.pickle
title_stem_index_directory/101_000.bin
title_stem_index_directory/101_posting_locs.pickle
title_stem_index_directory/102_000.bin
title_stem_index_directory/102_posting_locs.pickle
title_stem_index_directory/103_000.bin
title_stem_index_directory/103_posting_locs.pickle
--------------------------------------------------------------------------------
Done
--------------------------------------------------------------------------------


In [36]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)

for blob in client.list_blobs(bucket_or_name='inverted_indexes_bucket',
                             prefix='title_stem_index_directory'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
PRINT('Done.')

--------------------------------------------------------------------------------
Done.
--------------------------------------------------------------------------------


In [37]:
# Create inverted index instance
inverted = InvertedIndex()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs

# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

# write the global stats out
inverted.write_index(base_dir='project_final_indexes', 
                     name='title_stem_index',
                     bucket_name='inverted_indexes_bucket')

PRINT('Done')

--------------------------------------------------------------------------------
Done
--------------------------------------------------------------------------------


### Visualize the Index ###

In [63]:
inverted_index = load_index(base_dir='project_final_indexes', 
                            name='title_stem_index',
                            bucket_name='inverted_indexes_bucket')

In [64]:
posting_locs = inverted_index.posting_locs
df_data = inverted_index.df

# Convert the posting_locs dictionary to a DataFrame
posting_locs_df = pd.DataFrame.from_dict(posting_locs, orient='index', columns=['Posting List'])

# Convert the df_data dictionary to a DataFrame
df_data_df = pd.DataFrame.from_dict(df_data, orient='index', columns=['Document Frequency'])

In [67]:
df_data_df_sorted = df_data_df.sort_values(by='Document Frequency', ascending=False)

In [68]:
PRINT(f'Data Frame shape -> {df_data_df_sorted.shape}')

--------------------------------------------------------------------------------
Data Frame shape -> (25230, 1)
--------------------------------------------------------------------------------


We can see that by using stemming we reduced the number of words from *26981* to *25230* !

In [69]:
df_data_df_sorted.head(4)

Document Frequency
list                 119863
footbal               81127
film                  62624
station               60826

## Generate Inverted Idex For Body & Id Pairs ##

In [90]:
parquetFile = spark.read.parquet(*paths)
body_id_pairs = parquetFile.select("text", "id").rdd

In [91]:
# time the index creation time
t_start = time()

# word counts map
word_counts = body_id_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered,
                                 base_dir_='body_index_directory',
                                 bucket_name_='inverted_indexes_bucket').collect()

index_const_time = time() - t_start

Exception in thread "serve RDD 101" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [92]:
PRINT(f'Total amount of time: {(index_const_time / 60):.2f} minutes')

--------------------------------------------------------------------------------
Total amount of time: 75.40 minutes
--------------------------------------------------------------------------------


In [93]:
blobs_test = client.list_blobs(bucket_or_name='inverted_indexes_bucket',
                              prefix='body_index_directory')
count = 0
# Iterate over the blobs
for blob in blobs_test:
    if count == 10:break
    count+=1
    print(blob.name)
    
PRINT('Done')

body_index_directory/0_000.bin
body_index_directory/0_001.bin
body_index_directory/0_002.bin
body_index_directory/0_003.bin
body_index_directory/0_004.bin
body_index_directory/0_005.bin
body_index_directory/0_006.bin
body_index_directory/0_007.bin
body_index_directory/0_008.bin
body_index_directory/0_009.bin
--------------------------------------------------------------------------------
Done
--------------------------------------------------------------------------------


In [94]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)

for blob in client.list_blobs(bucket_or_name='inverted_indexes_bucket',
                             prefix='body_index_directory'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
PRINT('Done.')

--------------------------------------------------------------------------------
Done.
--------------------------------------------------------------------------------


In [95]:
# Create inverted index instance
inverted = InvertedIndex()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs

# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

# write the global stats out
inverted.write_index(base_dir='project_final_indexes', 
                     name='body_index',
                     bucket_name='inverted_indexes_bucket')

PRINT('Done')

--------------------------------------------------------------------------------
Done
--------------------------------------------------------------------------------


## Generate Inverted Index for Body & ID Pairs using Stemming ##

In [100]:
# time the index creation time
t_start = time()

# word counts with stemming map
word_counts_with_stemming = body_id_pairs.flatMap(lambda x: word_count_with_stemming(x[0], x[1]))
postings = word_counts_with_stemming.groupByKey().mapValues(reduce_word_counts)

# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered,
                                 base_dir_='body_stem_index_directory',
                                 bucket_name_='inverted_indexes_bucket').collect()

index_const_time = time() - t_start

ERROR:root:KeyboardInterrupt while sending command.              (37 + 5) / 124]
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/miniconda3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
PRINT(f'Total amount of time: {(index_const_time / 60):.2f} minutes')

In [ ]:
blobs_test = client.list_blobs(bucket_or_name='inverted_indexes_bucket',
                              prefix='body_stem_index_directory')
count = 0
# Iterate over the blobs
for blob in blobs_test:
    if count == 10:break
    count+=1
    print(blob.name)
    
PRINT('Done')

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)

for blob in client.list_blobs(bucket_or_name='inverted_indexes_bucket',
                             prefix='body_stem_index_directory'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
PRINT('Done.')

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs

# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

# write the global stats out
inverted.write_index(base_dir='project_final_indexes', 
                     name='body_stem_index',
                     bucket_name='inverted_indexes_bucket')

PRINT('Done')

## Generate Document Lenghs for Body & Title ##

### Generate Body Document Length Dictionary ###

In [40]:
parquetFile = spark.read.parquet(*paths)
body_id_rdd = parquetFile.select("text", "id").rdd

In [41]:
body_dictionary_lenght = {} # maps (key,value)=(doc_id, body_doc_lenght)

In [45]:
from sklearn.preprocessing import normalize
import numpy as np

# Example 1D array
array_1d = np.array([1, 2, 3, 4, 5])

# Reshape the 1D array into a 2D array with one row
array_2d = array_1d.reshape(1, -1)

# Normalize the 2D array along axis 1 (row-wise) using L2 normalization
normalized_array = normalize(array_2d, norm='l2', axis=1)

# Get the normalized value (a single number)
normalized_value = normalized_array[0, 0]

print("Normalized value:", normalized_value)

Normalized value: 0.13483997249264842


In [46]:
def count_unique_words(text):
    # Split the text into words
    words = text.split()
    
    # Count the occurrences of each word
    word_counts = {}
    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1
    
    # Get the counts of unique words
    unique_word_counts = list(word_counts.values())
    
    return unique_word_counts

text = 'pizza pizza pasta'
result = count_unique_words(text)
print(result)

[2, 1]


In [48]:
def calculate_doc_length(row):
    print('hi')
    doc_id = row['id']
    text = row['text']
    text_tok = tokenize(text, True)
    print(text_tok)
    return (doc_id, len(text_tok))

# Apply the function to each row of the RDD and map it to (doc_id, doc_length) pairs
doc_lengths_rdd = body_id_rdd.map(calculate_doc_length)

# Collect the results as a dictionary
body_dictionary_lenght = dict(doc_lengths_rdd.collect())

ERROR:root:KeyboardInterrupt while sending command.][Stage 18:> (0 + 0) / 124]
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/miniconda3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [18]:
base_dir='project_final_indexes' 
dl_name='body_dictionary_length'
bucket_name='inverted_indexes_bucket'

In [30]:
def get_bucket(bucket_name):
    return storage.Client('irprojectilayvictor').bucket(bucket_name)

In [18]:
path = str(Path(base_dir) / f'{dl_name}.pkl')
bucket = None if bucket_name is None else get_bucket(bucket_name)

Path(base_dir).mkdir(parents=True, exist_ok=True)

blob = bucket.blob(path)
pickle.dump(body_dictionary_lenght, open(path, 'wb'))
blob.upload_from_filename(path)

### Verify that the Dictionary Saved Successfully ###

In [12]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Download the pickled file from the bucket
blob = bucket.blob(path)
blob.download_to_filename(path)

with open(path, 'rb') as f:
    loaded_dict = pickle.load(f)

In [13]:
for key, value in list(loaded_dict.items())[:3]:
    PRINT(f"Key: {key}, Value: {value}")

--------------------------------------------------------------------------------
Key: 4045403, Value: 14477
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045413, Value: 2069
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045419, Value: 5353
--------------------------------------------------------------------------------


### Generate Title Document Lenght Dictionary ###

In [10]:
parquetFile = spark.read.parquet(*paths)
title_id_rdd = parquetFile.select("title", "id").rdd

In [11]:
title_dictionary = {} # maps (key, value)=(doc_id, title)

In [12]:
def calculate_title(row):
    doc_id = row['id']
    title = row['title']
    return (doc_id, title)

# Apply the function to each row of the RDD and map it to (doc_id, title) pairs
title_rdd = title_id_rdd.map(calculate_title)

title_dictionary = dict(title_rdd.collect())

In [12]:
for key, value in list(title_dictionary.items())[:3]:
    PRINT(f"Key: {key}, Value: {value}")

--------------------------------------------------------------------------------
Key: 4045403, Value: Foster Air Force Base
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045413, Value: Torino Palavela
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045419, Value: Mad About the Boy
--------------------------------------------------------------------------------


In [19]:
base_dir='project_final_indexes' 
dl_name='doc_norm'
bucket_name='inverted_indexes_bucket'

In [22]:
path = str(Path(base_dir) / f'{dl_name}.pkl')
bucket = None if bucket_name is None else get_bucket(bucket_name)

Path(base_dir).mkdir(parents=True, exist_ok=True)

blob = bucket.blob(path)
pickle.dump(doc_norm, open(path, 'wb'))
blob.upload_from_filename(path)

NameError: name 'doc_norm' is not defined

### Verify that the Dictionary Saved Successfully ###

In [26]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Download the pickled file from the bucket
blob = bucket.blob(path)
blob.download_to_filename(path)

with open(path, 'rb') as f:
    loaded_dict = pickle.load(f)

In [27]:
for key, value in list(loaded_dict.items())[:3]:
    PRINT(f"Key: {key}, Value: {value}")

--------------------------------------------------------------------------------
Key: 4045403, Value: 21
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045413, Value: 15
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 4045419, Value: 17
--------------------------------------------------------------------------------


## Execute PageRank & Save ##

In [23]:
def generate_graph(pages):
  ''' 
  Compute the directed graph generated by wiki links.
  Parameters: An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns: 
   - edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
   - vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  edges_w_dup = pages.flatMap(lambda page: [(page[0],anchor[0]) for anchor in page[1]])
  edges = edges_w_dup.distinct()
  vertices = edges.flatMap(lambda x: x)
  vertices = vertices.distinct()
  vertices = vertices.map(lambda x: Row(x)) #converting entries to a format that fits toDF() func up next
  return edges, vertices

In [26]:
t_start = time()

pages_links = parquetFile.select("id", "anchor_text").rdd

# construct the graph 
edges, vertices = generate_graph(pages_links)

# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')

g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)

pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())

pr_time = time() - t_start

24/02/25 08:20:31 WARN YarnAllocator: Container from a bad node: container_1708846121196_0001_01_000005 on host: cluster-fb6c-w-2.c.irass3-413914.internal. Exit status: 143. Diagnostics: [2024-02-25 08:20:31.236]Container killed on request. Exit code is 143
[2024-02-25 08:20:31.236]Container exited with a non-zero exit code 143. 
[2024-02-25 08:20:31.236]Killed by external signal
.
24/02/25 08:20:31 ERROR YarnScheduler: Lost executor 4 on cluster-fb6c-w-2.c.irass3-413914.internal: Container from a bad node: container_1708846121196_0001_01_000005 on host: cluster-fb6c-w-2.c.irass3-413914.internal. Exit status: 143. Diagnostics: [2024-02-25 08:20:31.236]Container killed on request. Exit code is 143
[2024-02-25 08:20:31.236]Container exited with a non-zero exit code 143. 
[2024-02-25 08:20:31.236]Killed by external signal
.
24/02/25 08:20:31 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 4 for reason Container from a bad node: container_1708846121196

In [27]:
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9913.728782160779|
|  10568| 5385.349263642038|
|  32927| 5282.081575765276|
|  30680|  5128.23370960412|
|5843419| 4957.567686263868|
|  68253| 4769.278265355157|
|  31717| 4486.350180548311|
|  11867|4146.4146509127695|
|  14533|3996.4664408855006|
| 645042| 3531.627089803743|
|  17867|3246.0983906041415|
|5042916| 2991.945739166178|
|4689264| 2982.324883041747|
|  14532| 2934.746829203171|
|  25391|   2903.5462235134|
|   5405| 2891.416329154635|
|4764461| 2834.366987332661|
|  15573|2783.8651181588384|
|   9316|2782.0396464137702|
|8569916|2775.2861918400154|
+-------+------------------+
only showing top 20 rows



In [33]:
pr_dict = pr.collect()
pr_dict = {row['id']: row['pagerank'] for row in pr_dict}

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [34]:
base_dir='project_pageRank' 
page_rank ='pageRank'
bucket_name='bucket_for_index'

In [35]:
path = str(Path(base_dir) / f'{page_rank}.pkl')
bucket = None if bucket_name is None else get_bucket(bucket_name)

Path(base_dir).mkdir(parents=True, exist_ok=True)

blob = bucket.blob(path)
pickle.dump(pr_dict, open(path, 'wb'))
blob.upload_from_filename(path)

In [37]:
for key, value in list(pr_dict.items())[:10]:
    PRINT(f"Key: {key}, doc page rank: {value}")

--------------------------------------------------------------------------------
Key: 3434750, doc page rank: 9913.728782160779
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 10568, doc page rank: 5385.349263642038
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 32927, doc page rank: 5282.081575765276
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 30680, doc page rank: 5128.23370960412
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Key: 5843419, doc page rank: 4957.567686263868
-------------------------------------------

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 29 
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag